# Microestructura y Sistemas de Trading
## Technical Analysis

- José Armando Melchor Soto


---

#### Librerias 


In [56]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import ta

sns.set(style="darkgrid")

In [57]:
df = pd.read_csv('aapl_5m_train.csv').dropna()
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 39160 entries, 0 to 39577
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Unnamed: 0  39160 non-null  int64  
 1   Timestamp   39160 non-null  int64  
 2   Gmtoffset   39160 non-null  int64  
 3   Datetime    39160 non-null  object 
 4   Open        39160 non-null  float64
 5   High        39160 non-null  float64
 6   Low         39160 non-null  float64
 7   Close       39160 non-null  float64
 8   Volume      39160 non-null  float64
dtypes: float64(5), int64(3), object(1)
memory usage: 3.0+ MB


#### TA

In [58]:
rsi_indicator = ta.momentum.RSIIndicator(df['Close'], window=20)
df['rsi']= rsi_indicator.rsi()

In [59]:
historic = df.dropna()
historic['buy_signal']= historic.rsi < 20 
historic['sell_signal']= historic.rsi > 80

C:\Users\ppmel\AppData\Local\Temp\ipykernel_10876\564761109.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  historic['buy_signal']= historic.rsi < 20
C:\Users\ppmel\AppData\Local\Temp\ipykernel_10876\564761109.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  historic['sell_signal']= historic.rsi > 80


In [60]:
historic.head()

,Unnamed: 0,Timestamp,Gmtoffset,Datetime,Open,High,Low,Close,Volume,rsi,buy_signal,sell_signal
19,19,1609776300,0,2021-01-04 16:05:00,130.309997,130.549896,129.660003,129.866699,3030766.0,26.683236,False,False
20,20,1609776600,0,2021-01-04 16:10:00,129.860000,130.239898,129.399993,130.089996,2231137.0,30.749593,False,False
21,21,1609776900,0,2021-01-04 16:15:00,130.080093,130.354995,129.904602,130.324996,1328068.0,34.758166,False,False
22,22,1609777200,0,2021-01-04 16:20:00,130.324005,130.419998,129.930099,129.996398,1233893.0,32.029264,False,False
23,23,1609777500,0,2021-01-04 16:25:00,129.994995,130.206695,129.880004,129.895004,1144809.0,31.232801,False,False


In [61]:
COM = 0.125 / 100
SL = 0.5 
TP = 0.5
BORROW_RATE = 0.25 / 100

cash = 1000000
n_shares = 50

In [62]:
from dataclasses import dataclass

@dataclass
class Operation:
    time: str
    price: float
    stop_loss: float
    take_profit: float
    n_shares: int
    type : str 

class Operation2:
    def __init__(self, time, price, stop_loss, take_profit, n_shares, type):
        self.time = time
        self.price = price
        self.stop_loss = stop_loss
        self.take_profit = take_profit
        self.n_shares = n_shares
        self.type = type
        


In [63]:
active_positions = []

for i, row in historic.iterrows():
    # Close position 

    for position in active_positions.copy():
        if row.Close > position.take_profit or row.Close < position.stop_loss:
            cash += row.Close * position.n_shares * (1 - COM)
            active_positions.remove(position)
            


    #-- Buy ---
    if not row.buy_signal:
        continue

    # Check Signal
    # Do we have enough cash?
    if cash < row.Close * n_shares * (1 + COM):
        continue

    # Discount the cost
    cash -= row.Close * n_shares * (1 + COM)

    # Save the operation as active position
    active_positions.append(
        Operation2(
            time = row.Datetime,
            price = row.Close,
            stop_loss = row.Close * (1 - SL),
            take_profit = row.Close * (1 + TP),
            n_shares = n_shares,
            type = 'LONG'
        )
    )

    



In [64]:
len(active_positions)

90

In [65]:
cash

385095.93201393756